In [0]:
print("Initializing pre-processing and queries fase...")

Initializing pre-processing and queries fase...


In [0]:
from pyspark.sql.functions import *

In [0]:
print("Verifying the shape of dataset")
df.count(), len(df.columns)

Verifying the shape of dataset


(1025, 14)

In [0]:
print("Gathering information")
display(df.describe())

Gathering information


summary,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025
mean,54.43414634146342,0.6956097560975609,0.9424390243902439,131.61170731707318,246.0,0.14926829268292682,0.5297560975609756,149.11414634146342,0.33658536585365856,1.0715121951219513,1.3853658536585365,0.7541463414634146,2.32390243902439,0.5131707317073171
stddev,9.072290233244276,0.46037332411964915,1.029640743645865,17.516718005376408,51.59251020618206,0.3565266897271576,0.527877566874892,23.00572374597721,0.47277237600371186,1.175053255150176,0.6177552671745917,1.030797665024282,0.6206602380510299,0.5000704980788013
min,29,0,0,94,126,0,0,71,0,0.0,0,0,0,0
max,77,1,3,200,564,1,2,202,1,6.2,2,4,3,1


In [0]:
print("Gathering more information")
display(df.summary())

Gathering more information


summary,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025
mean,54.43414634146342,0.6956097560975609,0.9424390243902439,131.61170731707318,246.0,0.14926829268292682,0.5297560975609756,149.11414634146342,0.33658536585365856,1.0715121951219513,1.3853658536585365,0.7541463414634146,2.32390243902439,0.5131707317073171
stddev,9.072290233244276,0.46037332411964915,1.029640743645865,17.516718005376408,51.59251020618206,0.3565266897271576,0.527877566874892,23.00572374597721,0.47277237600371186,1.175053255150176,0.6177552671745917,1.030797665024282,0.6206602380510299,0.5000704980788013
min,29,0,0,94,126,0,0,71,0,0.0,0,0,0,0
25%,48,0,0,120,211,0,0,132,0,0.0,1,0,2,0
50%,56,1,1,130,240,0,1,152,0,0.8,1,0,2,1
75%,61,1,2,140,275,0,1,166,1,1.8,2,1,3,1
max,77,1,3,200,564,1,2,202,1,6.2,2,4,3,1


In [0]:
print("Checking for null values")
null_counts = df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in df.columns])
display(null_counts)


Checking for null values


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
print("Droping duplicated values")
df = df.dropDuplicates()

Droping duplicated values


In [0]:
print("Adding categorical values for sex column")
df = df.withColumn("descritive_sex", when(col("sex") == 0, "Feminino").otherwise("Masculino"))

Adding categorical values for sex column


In [0]:
print("Changing numeric values for descritive ones for target column")
df = df.withColumn("target", when(col("target") == 1, "Tendência de ataque do coração").otherwise("Sem tendência de ataque do coração"))

Changing numeric values for descritive ones for target column


In [0]:
print("Updating table heart after drop duplicates")
df.createOrReplaceTempView("heart")

Updating table heart after drop duplicates


### Perguntas

#### Q1. Qual a distribuição entre pessoas do sexo masculino e feminino?

In [0]:
%sql
SELECT 
    CASE 
        WHEN sex = 0 THEN "Feminino"
        ELSE "Masculino"
    END AS SEX, 
    COUNT(*) AS TOTAL,
    ROUND((COUNT(*) / (SELECT COUNT(*) FROM heart)) * 100,2) AS PERCENTAGE FROM heart 
GROUP BY sex

SEX,TOTAL,PERCENTAGE
Feminino,96,31.79
Masculino,206,68.21


In [0]:
# Código em PySpark (DataFrames)

total = df.count()

q1 = df.groupBy("sex") \
       .agg(
            count("*").alias("total"),
            round(count("*") / total * 100, 2).alias("percentage")
           ) \
       .withColumn("sex", when(col("sex") == 0, "Feminino").otherwise("Masculino"))

#### Q2. Qual a quantidade entre pessoas do sexo masculino e feminino de apresentarem tendência de ataque do coração?

In [0]:
%sql
SELECT 
  CASE
    WHEN sex = 0 THEN "Feminino"
    ELSE "Masculino"
  END AS SEX, target AS DIAGNOSTIC, COUNT(*) AS TOTAL
FROM heart 
GROUP BY sex, target

SEX,DIAGNOSTIC,TOTAL
Feminino,Tendência de ataque do coração,72
Feminino,Sem tendência de ataque do coração,24
Masculino,Sem tendência de ataque do coração,114
Masculino,Tendência de ataque do coração,92


In [0]:
# Código em PySpark (DataFrames)
q2 = df.groupBy("sex", "target") \
       .count() \
       .withColumnRenamed("target", "diagnostic") \
       .withColumnRenamed("count", "total") \
       .withColumn("sex", when(col("sex") == 0, "Feminino").otherwise("Masculino"))

#### Q3. Qual a quantidade da tendência de ataque do coração por tipo de dor no peito?

In [0]:
%sql
SELECT cp AS TIPO_DOR, COUNT(target) AS TOTAL
FROM heart 
WHERE target = 'Tendência de ataque do coração'
GROUP BY cp
ORDER BY TOTAL DESC

TIPO_DOR,TOTAL
2,68
1,41
0,39
3,16


In [0]:
# Código em PySpark (DataFrames)
q3 = df.filter(col("target") == "Tendência de ataque do coração") \
       .groupBy("cp") \
       .count() \
       .withColumnRenamed("cp", "tipo_dor") \
       .withColumnRenamed("count", "total") \
       .orderBy(desc("total"))

#### Q4. Qual a quantidade de pessoas do sexo masculino e feminino por nível de talassemia? 

In [0]:
%sql
SELECT 
  CASE 
    WHEN sex = 0 THEN "Feminino"
    ELSE "Masculino"
  END AS SEXO, thal AS TALASSEMIA, COUNT(*) AS TOTAL
FROM heart
GROUP BY sex, thal
ORDER BY TALASSEMIA, TOTAL

SEXO,TALASSEMIA,TOTAL
Masculino,0,1
Feminino,0,1
Feminino,1,1
Masculino,1,17
Feminino,2,79
Masculino,2,86
Feminino,3,15
Masculino,3,102


In [0]:
# Código em PySpark (DataFrames)
q4 = df.groupBy("sex", "thal") \
       .count() \
       .withColumn("sex", when(col("sex") == 0, "Feminino").otherwise("Masculino")) \
       .withColumnRenamed("thal", "talassemia") \
       .withColumnRenamed("count", "total") \
       .orderBy("talassemia", "total")

#### Q5. Qual a quantidade da tendência de ataque do coração por slope?

In [0]:
%sql
SELECT slope AS SLOPE, target AS DIAGNOSTIC, COUNT(*) AS TOTAL
FROM heart
GROUP BY SLOPE, DIAGNOSTIC
ORDER BY SLOPE, TOTAL

SLOPE,DIAGNOSTIC,TOTAL
0,Tendência de ataque do coração,9
0,Sem tendência de ataque do coração,12
1,Tendência de ataque do coração,49
1,Sem tendência de ataque do coração,91
2,Sem tendência de ataque do coração,35
2,Tendência de ataque do coração,106


In [0]:
# Código em PySpark (DataFrames)
q5 = df.groupBy("slope", "target") \
       .count() \
       .withColumnRenamed("target", "diagnostic") \
       .withColumnRenamed("count", "total") \
       .orderBy("slope", "total")

#### Q6. Quantas pessoas do sexo masculino e do sexo feminino tem entre 30 e 50 anos?

In [0]:
%sql
SELECT 
  CASE 
    WHEN sex = 0 THEN "Feminino"
    ELSE "Masculino"
  END AS SEX,
  COUNT(*) AS AGE_GROUP
FROM heart 
WHERE age BETWEEN 30 AND 50
GROUP BY sex

SEX,AGE_GROUP
Feminino,27
Masculino,66


In [0]:
# Código em PySpark (DataFrames)
q6 = df.filter(col("age").between(30, 50)) \
       .groupBy("sex") \
       .count() \
       .withColumn("sex", when(col("sex") == 0, "Feminino").otherwise("Masculino")) \
       .withColumnRenamed("count", "age_group")

#### Q7. Quantas pessoas, por sexo, abaixo dos 50 anos possem tendência de um ataque do coração?

In [0]:
%sql
SELECT 
  CASE
    WHEN sex = 0 THEN "Feminino"
    ELSE "Masculino"
  END AS SEX,
  COUNT(*) AS TOTAL
FROM heart 
WHERE age < 50 AND target = 'Tendência de ataque do coração'
GROUP BY SEX

SEX,TOTAL
Feminino,23
Masculino,38


In [0]:
# Código em PySpark (DataFrames)
q7 = df.filter((col("age") < 50) & (col("target") == "Tendência de ataque do coração")) \
       .groupBy("sex") \
       .count() \
       .withColumn("sex", when(col("sex") == 0, "Feminino").otherwise("Masculino")) \
       .withColumnRenamed("count", "total")

#### Q8. Qual a média de idade das mulheres entrevistadas que não possuem tendência de sofrer de um ataque do coração?

In [0]:
%sql
SELECT ROUND(MEAN(age),2) AS AVERAGE_AGE
FROM heart 
WHERE sex = 0 AND target <> 'Tendência de ataque do coração'

AVERAGE_AGE
59.04


In [0]:
# Código em PySpark (DataFrames)
q8 = df.filter((col("sex") == 0) & (col("target") != "Tendência de ataque do coração")) \
       .select(round(mean("age"), 2).alias("average_age"))

#### Q9. Qual a idade da pessoa mais nova e mais velha do conjunto de dados, por sexo?

In [0]:
%sql
SELECT 
  CASE 
    WHEN sex = 0 THEN "Feminino"
    ELSE "Masculino"
  END AS SEX, 
  MIN(age) PESSOA_MAIS_NOVO, 
  MAX(age) PESSOA_MAIS_VELHA
FROM heart 
GROUP BY sex

SEX,PESSOA_MAIS_NOVO,PESSOA_MAIS_VELHA
Feminino,34,76
Masculino,29,77


In [0]:
# Código em PySpark (DataFrames)
q9 = df.groupBy("sex") \
       .agg(
           min("age").alias("pessoa_mais_nova"),
           max("age").alias("pessoa_mais_velha")
       ) \
       .withColumn("sex", when(col("sex") == 0, "Feminino").otherwise("Masculino"))

#### Q10. Quantas pessoas, por faixa etária, com colesterol acima de 300 apresentam tendência de um ataque do coração? 

In [0]:
%sql
SELECT 
  CASE 
    WHEN age < 20 THEN '0 - 19'
    WHEN age < 30 THEN '20 - 29'
    WHEN age < 40 THEN '30 - 39'
    WHEN age < 50 THEN '40 - 49'
    WHEN age < 60 THEN '50 - 59'
    WHEN age < 70 THEN '60 - 69'
    WHEN age < 80 THEN '70 - 79'
    WHEN age < 90 THEN '80 - 89'
  END AS AGE_GROUP,
  COUNT(*) AS TOTAL
FROM heart 
WHERE chol > 300 AND target = 'Tendência de ataque do coração'
GROUP BY AGE_GROUP
ORDER BY AGE_GROUP


AGE_GROUP,TOTAL
30 - 39,1
40 - 49,4
50 - 59,8
60 - 69,9
70 - 79,1


In [0]:
# Código em PySpark (DataFrames)
q10 = df.filter((col("chol") > 300) & (col("target") == "Tendência de ataque do coração")) \
        .withColumn("age_group", when(col("age") < 20, "0 - 19") \
                           .when(col("age") < 30, "20 - 29") \
                           .when(col("age") < 40, "30 - 39") \
                           .when(col("age") < 50, "40 - 49") \
                           .when(col("age") < 60, "50 - 59") \
                           .when(col("age") < 70, "60 - 69") \
                           .when(col("age") < 80, "70 - 79") \
                           .when(col("age") < 90, "80 - 89")
                   ) \
        .groupBy("age_group") \
        .count().alias("total") \
        .orderBy("age_group")

In [0]:
print("Finishing pre-processing and queries fase and closing silver layer")

Finishing pre-processing and queries fase and closing silver layer
